In [4]:
#This class creates the image features
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tqdm import tqdm
import json

In [5]:
import tensorflow as tf
# Configure GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

import requests
from PIL import Image
from io import BytesIO
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tqdm import tqdm

def download_and_extract_image_features(items):
    # Load ResNet50 model
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    
    # Initialize features list with zeros for all items
    features = [np.zeros(2048) for _ in range(len(items))]
    
    for idx, item in tqdm(enumerate(items), total=len(items), desc="Processing images"):
        if 'images' in item and item['images']:
            image_url = item['images'][0]  # Get first image URL
            try:
                # Download image
                response = requests.get(image_url, timeout=10)
                if response.status_code == 200:
                    # Open image
                    img = Image.open(BytesIO(response.content))
                    
                    # Preprocess the image
                    img = img.resize((224, 224))  # ResNet50 expected size
                    img = img.convert('RGB')
                    x = image.img_to_array(img)
                    x = np.expand_dims(x, axis=0)
                    x = preprocess_input(x)
                    
                    # Extract features
                    feat = model.predict(x, verbose=0)
                    features[idx] = feat.flatten()  # Store at correct index
            except Exception as e:
                print(f"Error downloading or processing image {idx}: {str(e)}")
                # No need to append zeros as the list is pre-initialized
    
    # Convert to numpy array
    features = np.array(features)
    print("Features shape:", features.shape)
    
    # Save features
    with open("../data/res_image_features.npy", "wb") as f:
        np.save(f, features)
    
    return features

# Example usage:
with open("../data/item_with_images.json", "r") as items_json:
    items = json.load(items_json)
features = download_and_extract_image_features(items)

Processing images: 100%|██████████| 3253/3253 [16:16<00:00,  3.33it/s]


Features shape: (3253, 2048)


In [6]:
with open("../data/item_with_images.json", "r") as items_json:
    items = json.load(items_json)
len(items)

3253